<a href="https://colab.research.google.com/github/RICQW/pdf-input-url/blob/main/clean_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install camelot-py

In [ ]:
import camelot

# 读取 PDF 表格
tables = camelot.read_pdf("https://cdn.jsdelivr.net/gh/RICQW/pdf-input-url/test1.pdf", pages="all")

# 初始化 result
result = []

row_counter = 1  # 连续行编号
num_cols = 6     # 根据你的表格列数修改

# 1️⃣ 读取并清理内容
for table in tables:
    df = table.df
    for r in range(len(df)):
        for c in range(len(df.columns)):
            cleaned = df.iloc[r, c].replace(" ", "").replace("\n", "")
            cell_id = f"R{row_counter}C{c+1}"
            result.append((cell_id, cleaned))
        row_counter += 1

# 2️⃣ 按行分组
rows = [result[i:i+num_cols] for i in range(0, len(result), num_cols)]

# 3️⃣ 处理序号行：合并下一行内容到上一行
i = 1  # 从第二行开始，跳过第一行
while i < len(rows) - 1:  # 保证有下一行
    current_row = rows[i]
    if any(cell[1] == "序号" for cell in current_row):
        prev_row = rows[i-1]
        next_row = rows[i+1]
        for j in range(len(current_row)):
            prev_row[j] = (prev_row[j][0], prev_row[j][1] + next_row[j][1])
        # 删除序号行和下一行
        rows.pop(i+1)
        rows.pop(i)
        continue  # 不增加 i，下一行已经移位
    i += 1

# 4️⃣ 重新编号
final_result = []
row_counter = 1
for row in rows:
    for c_idx, cell in enumerate(row):
        cell_id = f"R{row_counter}C{c_idx+1}"
        final_result.append((cell_id, cell[1]))
    row_counter += 1

# 5️⃣ 输出最终结果
print(final_result)


In [ ]:
import csv

with open("result.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_result)
